In [1]:
import pickle
from glob import glob
import numpy as np
import os
from tqdm import tqdm

In [2]:
pkl_list = glob(os.path.join('train', '*'))
len(pkl_list)

205942

In [3]:
X = []
y = []

for pkl_path in tqdm(pkl_list):
    with open(pkl_path, 'rb') as f:
        scene = pickle.load(f)
    
    # the index of agent to be predicted 
    pred_id = np.where(scene["track_id"] == scene['agent_id'])[0][0]
    
    p_in_raw = scene['p_in'][pred_id]
    p_out_raw = scene['p_out'][pred_id]
    v_in_raw = scene['v_in'][pred_id]
    lane_scene = scene['lane']
    
    # Normalization
    min_vecs = np.min(lane_scene, axis = 0)
    max_vecs = np.max(lane_scene, axis = 0)
    
    # Normalize by vectors
    p_in_normalized = (p_in_raw - min_vecs)/(max_vecs - min_vecs)
    p_out_normalized = (p_out_raw - min_vecs)/(max_vecs - min_vecs)
    v_in_norm = np.linalg.norm(v_in_raw, axis=1, keepdims=True)
    v_in_norm = np.where(v_in_norm == 0.0, 1.0, v_in_norm)
    v_in_normalized = v_in_raw / v_in_norm
    inp = np.concatenate((p_in_normalized,v_in_normalized),axis=1)

    X.append(inp.T)
    y.append(p_out_normalized.flatten())

100%|██████████| 205942/205942 [16:39<00:00, 206.07it/s]


In [4]:
npx = np.stack(X)
npx.shape

(205942, 4, 19)

In [5]:
npy = np.vstack(y)
npy.shape

(205942, 60)

In [6]:
with open('train_X_cnn.npy', 'wb') as f:
    np.save(f, npx)

with open('train_y_cnn.npy', 'wb') as f:
    np.save(f, npy)